In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils
import keras.losses
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras.layers.noise import GaussianNoise
from keras.callbacks import ModelCheckpoint
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from time import time
import itertools

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Using TensorFlow backend.


In [19]:
def parse_data(file):
    # Import csv file
    raw = pd.read_csv(file)
    train = raw[raw['Usage'] == 'Training']
    train_pixels = train['pixels']
    train_y = train['emotion'].values
    
    test = raw[raw['Usage'] == 'PublicTest']
    test_pixels = test['pixels']
    test_y = test['emotion'].values
    
    # Convert raw training pixel data into numpy array
    train_x = []
    for image in train_pixels:
        temp = [int(n) for n in image.split()]
        train_x.append(temp)
    train_x = np.asarray(train_x)
    train_x = np.resize(train_x, (len(train_pixels),1, 48, 48))
    
    # Convert raw testing pixel data into numpy arrays
    test_x = []
    for image in test_pixels:
        temp = [int(n) for n in image.split()]
        test_x.append(temp)
    test_x = np.asarray(test_x)
    test_x = np.resize(test_x, (len(test_pixels), 1, 48, 48))
    
    return train_x, train_y, test_x, test_y



In [ ]:
train_x, train_y, test_x, test_y = parse_data('fer2013.csv')

In [ ]:
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

In [5]:
weights = class_weight.compute_class_weight('balanced', np.unique(train_y), train_y)
train_y = np_utils.to_categorical(train_y,10)

In [8]:
# Use image augmentation
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    data_format='channels_first')

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(train_x)

In [10]:
in_shape = (1, 48, 48)
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same', data_format='channels_first', activation='relu', input_shape=in_shape))
model.add(MaxPooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(GaussianNoise(0.3))

model.add(Conv2D(128, kernel_size=(5,5), strides=(1,1), padding='same', data_format='channels_first', activation='relu', input_shape=in_shape))
model.add(MaxPooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(512, kernel_size=(3,3), strides=(1,1), padding='same', data_format='channels_first', activation='relu'))
model.add(MaxPooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(GaussianNoise(0.3))

model.add(Conv2D(512, kernel_size=(3,3), strides=(1,1), padding='same', data_format='channels_first', activation='relu'))
model.add(MaxPooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.50))

model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.50))

model.add(Dense(7, activation='softmax'))

In [11]:
# load weights
# Uncomment below to restore weights if kernel was interrupted during training 
#model.load_weights("weights.hdf5")

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])

filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')

In [9]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(train_x, train_y, batch_size=32), 
                    steps_per_epoch=len(train_x) / 32, epochs=30, class_weight=weights, callbacks=[checkpoint])
# Normal model fit
model.fit(train_x, train_y, epochs=50, batch_size=30, verbose = 1, class_weight=weights, callbacks=[checkpoint])

Epoch 1/10
28709/28709 [==============================] - 83s 3ms/step - loss: 0.4584 - acc: 0.8410

Epoch 00001: acc improved from -inf to 0.84103, saving model to weights-best.hdf5
Epoch 2/10
28709/28709 [==============================] - 80s 3ms/step - loss: 0.2594 - acc: 0.9106

Epoch 00002: acc improved from 0.84103 to 0.91055, saving model to weights-best.hdf5
Epoch 3/10
28709/28709 [==============================] - 80s 3ms/step - loss: 0.2194 - acc: 0.9265

Epoch 00003: acc improved from 0.91055 to 0.92647, saving model to weights-best.hdf5
Epoch 4/10
28709/28709 [==============================] - 80s 3ms/step - loss: 0.2126 - acc: 0.9280

Epoch 00004: acc improved from 0.92647 to 0.92797, saving model to weights-best.hdf5
Epoch 5/10
28709/28709 [==============================] - 80s 3ms/step - loss: 0.2189 - acc: 0.9263

Epoch 00005: acc did not improve
Epoch 6/10
28709/28709 [==============================] - 80s 3ms/step - loss: 0.2087 - acc: 0.9301

Epoch 00006: acc improve

In [12]:
predictions = model.predict(test_x)

In [13]:
y_hat = []
for value in predictions:
    y_hat.append(np.argmax(value))
    
y_hat = np.array(y_hat)

In [ ]:
print(classification_report(test_y,y_hat))